In [2]:
import os
import sys
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.insert(0, parent_directory)
import time
import torch
import esm
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from model import MMCLKins
from torch_geometric.data import Data
from process_3dkdavis import generate_inner_coor, generate_protein_sequence, generate_graph_feature, gen_seq_list, gene_smi_fes
from collates import collate


#### Downloading the datasets to main directory
##### https://fca_icdb.mpu.edu.mo/DataResource/database/virtual_screening/crk12.tar.gz

Running the following script under main directory:
> tar zxvf crk12.tar.gz -C ./cases

#### Downloading the pkl file to main directory
##### https://fca_icdb.mpu.edu.mo/DataResource/database/pkls/virtual_screening.tar.gz

Running the following script under main directory:
> tar zxvf virtual_screening.tar.gz -C ./pkls

#### Uploading the dataset

In [3]:
ligand_path = '../cases/crk12/jmc-Q382V0-sdfs'
kinase_path = '../cases/crk12/Q382V0-kinase.pdb'
pocket_path = '../cases/crk12/Q382V0-kinase_pocket_14.pdb'

#### Extracting the features

In [4]:
def generate_kin_fes(ki, batch_converter, model1):
    error_pdbs = []
    pdbs_1022 = []
    error_num = []
    kinase_feas = {}
    kina = ki.split('.p')[0]
    pdb_path = ki
    pro_index = gen_seq_list(pdb_path)
    protein, protein_graph, coords = generate_graph_feature(pdb_path)
    protein_sequence, chain_num = generate_protein_sequence(pdb_path)
    
    pr_dist, pr_theta, pr_phi, pr_tau = generate_inner_coor(protein.x, protein.node_s, protein.edge_index)
    prot = [(0, protein_sequence[:1022])]
    batch_labels, batch_strs, batch_tokens = batch_converter(prot)  #0（蛋白序号）；Fasta序列，蛋白表征
    with torch.no_grad():
        results = model1(batch_tokens, repr_layers=[33], return_contacts=True)
    pro_token_repre = results["representations"][33]
    n = 0
    if pro_token_repre.shape[1] != protein.node_s.shape[0] + 2 :
        n = n + 1
        pdbs_1022.append(ki)
        error_num.append(pro_token_repre.shape[1] - protein.node_s.shape[0] - 2)
        if pro_token_repre.shape[1] > 1022:
            print(f'{ki} pdb exceeds 1022!!!')
        else:
            error_pdbs.append(ki)
            print(f'{ki} pdb exceeds 1022!!!')
        
    kinase_fea = Data(pro_graphs=protein_graph,
                    pro_index=pro_index,
                    pro_atoms_feats_s=protein.node_s,
                    pro_atoms_feats_v=protein.node_v,
                    pro_coords_feats=protein.x,
                    pro_edges_feats_s=protein.edge_s,
                    pro_edges_feats_v=protein.edge_v,
                    pro_edge_index=protein.edge_index,
                    pro_token_repre=pro_token_repre,
                    pro_fp=protein_sequence,
                    pr_dist=pr_dist,
                    pr_theta=pr_theta,
                    pr_phi=pr_phi,
                    pr_tau=pr_tau)
    kinase_feas[kina] = kinase_fea
            
    return kinase_feas, pdbs_1022, error_pdbs

In [5]:
model1, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()
kinase_fes, kina_1022, error_kina = generate_kin_fes(kinase_path, batch_converter, model1)
pocket_fes, pock_1022, error_pock = generate_kin_fes(pocket_path, batch_converter, model1)

model_name = "DeepChem/ChemBERTa-10M-MLM"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
lig_fes, error_mol = gene_smi_fes(ligand_path, model, tokenizer)

#### Uploading the pkl file of MMCLKin model

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
path = '../pkls/virtual_screening/MMCLKin_DTI_pearson_best.pkl'

lstm_dropout = 0.2
alpha = 0.2
num_heads = 2
hidden_dim = 256
dropout_rate = 0.3
model = MMCLKins(lstm_dropout, alpha, num_heads, hidden_dim, dropout_rate, n_head=8, smile_vocab=63, local_rank=device) 
model.load_state_dict(torch.load(path)['model'], strict=True)
model.to(device)

MMCLKins(
  (dropout): Dropout(p=0.3, inplace=False)
  (leakyrelu): LeakyReLU(negative_slope=0.2)
  (relu): ReLU()
  (prelu): PReLU(num_parameters=1)
  (elu): ELU(alpha=1.0)
  (conv): GCNConv(259, 256)
  (conv3): GCNConv(256, 256)
  (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (bilstm): LSTM(256, 256, dropout=0.2, bidirectional=True)
  (self_attention): A_MultiHeadAttention(
    (qkv_proj): Linear(in_features=512, out_features=1536, bias=True)
    (o_proj): Linear(in_features=512, out_features=512, bias=True)
  )
  (out_fc): Linear(in_features=512, out_features=256, bias=True)
  (out_m_fc): Linear(in_features=256, out_features=128, bias=True)
  (output_fc): Linear(in_features=128, out_features=1, bias=True)
  (smi_egnn): EGNNConv(
    (edge_mlp): Sequential(
      (0): Linear(in_features=57, out_features=256, bias=True)
      (1): SiLU()
      (2): Linear(in_features=256, out_features=256, b

#### Predicting the binding affinity between kinase and inhibitors

In [7]:
model.eval()
pred_results = []
lig_keys = lig_fes.keys()
with torch.no_grad():
    c = 0
    for i, na in enumerate(lig_keys):
        lig_feas = lig_fes[na]
        kin_fes = kinase_fes['../cases/crk12/Q382V0-kinase']
        poc_fes = pocket_fes['../cases/crk12/Q382V0-kinase_pocket_14']
        x_feats = collate(lig_feas, kin_fes, poc_fes)
        x_feats = x_feats.to(device)
        y_pred, g_spo_att, g_spr_att, s_spo_att, s_spr_att, pg_spo_att, pg_spr_att, ps_spo_att, ps_spr_att, atominmol_indexes, subwinsmi_indexes, pocinpro_indexes = model(x_feats)
        re = [na, y_pred.item(), g_spo_att.tolist(), g_spr_att.tolist(), s_spo_att.tolist(), s_spr_att.tolist(), atominmol_indexes[0].tolist(), 
            pg_spo_att.tolist(), pg_spr_att.tolist(), subwinsmi_indexes[0].tolist(), ps_spo_att.tolist(), ps_spr_att.tolist(), pocinpro_indexes[0].tolist()]
        pred_results.append(re)
        
name = ['compound_id', 'pred_results', 'graph=smi+pocket_attention', 'graph=smi+pocket_in_protein_attention',      
            'sequence=smi+pocket_attention', 'sequence=smi+pocket_in_protein_attention','graph=smi_atominmol_indexes',
            'graph=atom_in_mol+pocket_attention(atominmol_indexes)', 'graph=atom_in_mol+protein_attention(atominmol_indexes)',
            'sequence=smi_subwinsmi_indexes', 'sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes)',
            'sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes)','pocinpro_indexes']
da = pd.DataFrame(columns=name, data=pred_results)
da

,compound_id,pred_results,graph=smi+pocket_attention,graph=smi+pocket_in_protein_attention,sequence=smi+pocket_attention,sequence=smi+pocket_in_protein_attention,graph=smi_atominmol_indexes,graph=atom_in_mol+pocket_attention(atominmol_indexes),graph=atom_in_mol+protein_attention(atominmol_indexes),sequence=smi_subwinsmi_indexes,sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes),sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes),pocinpro_indexes
0,C66200633_P101820074_1,5.131211,"[[-1.071001648902893, 1.0554227828979492, 1.24...","[[-1.2708230018615723, 1.1316097974777222, 1.2...","[[1.6654627323150635, -0.5504493117332458, 0.8...","[[1.52498197555542, -0.35180774331092834, 0.84...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-1.071001648902893, 1.0554227828979492, 1.24...","[[-1.9405150413513184, 1.6282575130462646, 1.8...","[0, 1, 2, 4, 5, 7, 9, 12, 16, 18, 20, 21, 22, ...","[[-0.5014050006866455, 0.916220486164093, 1.42...","[[-0.4918437600135803, 1.2569843530654907, 1.8...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17..."
1,C68568500_P105223047_10,4.969146,"[[-1.2004165649414062, 0.8747571110725403, 1.3...","[[-1.539724588394165, 0.9221487641334534, 1.37...","[[0.18834355473518372, -1.129435658454895, 0.6...","[[0.3420292139053345, -0.8457189798355103, 0.8...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-1.2004165649414062, 0.8747571110725403, 1.3...","[[-2.3228211402893066, 1.3527003526687622, 2.0...","[0, 1, 2, 4, 5, 7, 9, 12, 14, 16, 18, 20, 21, ...","[[-1.1271090507507324, 0.692938506603241, 0.88...","[[-1.2693085670471191, 1.0610854625701904, 1.1...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17..."
2,C40729987_P61844298_3,4.916657,"[[0.6650235056877136, -1.241023302078247, 0.24...","[[0.7888718843460083, -1.563041090965271, 0.24...","[[-1.4176968336105347, 0.16376259922981262, -1...","[[-1.0453062057495117, 0.34257981181144714, -1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.6650235056877136, -1.241023302078247, 0.24...","[[0.9646813273429871, -2.5452282428741455, 0.1...","[0, 1, 3, 6, 8, 9, 10, 12, 13, 14, 15, 17, 19,...","[[0.11105503886938095, -1.8402799367904663, 0....","[[0.2838352620601654, -2.223104953765869, 0.18...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17..."
3,C39936967_P60598514,4.994203,"[[0.429718017578125, -1.6805751323699951, -0.4...","[[0.43167921900749207, -1.9654638767242432, -0...","[[-1.278029203414917, 0.534161388874054, -1.80...","[[-1.0180739164352417, 0.6401610374450684, -1....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.429718017578125, -1.6805751323699951, -0.4...","[[0.6079286336898804, -3.1146364212036133, -0....","[0, 1, 2, 3, 5, 6, 8, 11, 13, 14, 16, 18, 19, ...","[[0.44604289531707764, -2.176889181137085, -0....","[[0.6269859671592712, -2.488065004348755, -1.1...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17..."
4,C53619608_P98898977_5,5.009614,"[[-0.30367326736450195, -1.9595056772232056, 1...","[[-0.41727834939956665, -2.289597511291504, 1....","[[-0.10449552536010742, -0.18062227964401245, ...","[[0.02076970785856247, -0.07776360958814621, -...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-0.30367326736450195, -1.9595056772232056, 1...","[[-0.6161109805107117, -3.3781321048736572, 1....","[0, 1, 2, 4, 5, 7, 10, 12, 13, 14, 15, 18, 20,...","[[-0.21361400187015533, -0.7828177809715271, 1...","[[-0.24111811816692352, -0.9871131181716919, 1...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,C36350565_P105904223,5.075810,"[[-0.6716181635856628, 0.6535751223564148, 1.2...","[[-0.817569375038147, 0.7127267718315125, 1.36...","[[0.38971689343452454, -1.2646222114562988, 0....","[[0.4723666310310364, -1.023592472076416, 0.85...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-0.6716181635856628, 0.6535751223564148, 1.2...","[[-1.3853533267974854, 0.9425452351570129, 1.9...","[0, 1, 2, 4, 5, 6, 7, 9, 11,

#### Processing the results

In [8]:
import operator
da['compound_id'] = da['compound_id'].str.replace('_P', '', regex=False)
df = da.sort_values(by='pred_results', ascending=False)
name = list(df['compound_id'])
label = 'posi'
all_le = 13
na = []
nas = []
nass = []
for i, ns in enumerate(name):
    if '_' in ns:
        if ns.split('_')[0] not in nas:
            nas.append(ns.split('_')[0])
            nass.append(df.loc[i].tolist())
    else:
        if ns not in nas:
            nas.append(ns)
            nass.append(df.loc[i].tolist())

all_length = len(nas)
m1 = m2 = m3 = m4 = m5 = m6 = m7 = m8 = m9= m10 = 0
for i, com in enumerate(nas):
    if i < all_length*0.01:
        if str(com[:4]) == label:
            m1 = m1 + 1
    if i < all_length*0.02:
        if str(com[:4]) == label:
            m2 = m2 + 1
    if i < all_length*0.03:
        if str(com[:4]) == label:
            m3 = m3 + 1
    if i < all_length*0.04:
        if str(com[:4]) == label:
            m4 = m4 + 1
    if i < all_length*0.05:
        if str(com[:4]) == label:
            m5 = m5 + 1
    if i < all_length*0.06:
        if str(com[:4]) == label:
            m6 = m6 + 1
    if i < all_length*0.07:
        if str(com[:4]) == label:
            m7 = m7 + 1
    if i < all_length*0.08:
        if str(com[:4]) == label:
            m8 = m8 + 1
    if i < all_length*0.09:
        if str(com[:4]) == label:
            m9 = m9 + 1
    if i < all_length*0.1:
        if str(com[:4]) == label:
            m10 = m10 + 1        

m1r = m1/(all_le)
m2r = m2/(all_le)
m3r = m3/(all_le)
m4r = m4/(all_le)
m5r = m5/(all_le)
m6r = m6/(all_le)
m7r = m7/(all_le)
m8r = m8/(all_le)
m9r = m9/(all_le)
m10r = m10/(all_le)

name = [ 'compound_id', 'pred_results', 'graph=smi+pocket_attention',
        'graph=smi+pocket_in_protein_attention','sequence=smi+pocket_attention',
        'sequence=smi+pocket_in_protein_attention',	'graph=smi_atominmol_indexes',
        'graph=atom_in_mol+pocket_attention(atominmol_indexes)','graph=atom_in_mol+protein_attention(atominmol_indexes)',
        'sequence=smi_subwinsmi_indexes', 'sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes)',	
        'sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes)', 'pocinpro_indexes']

new_csv = pd.DataFrame(columns=name, data=nass)
new_csv = pd.concat([new_csv, pd.Series(f'the number of the active drugs in top 1% ::: {m1} / {all_le} == {m1r}'), 
                    pd.Series(f'the number of the active drugs in top 2% ::: {m2} / {all_le} == {m2r}'), 
                    pd.Series(f'the number of the active drugs in top 3% ::: {m3} / {all_le} == {m3r}'), 
                    pd.Series(f'the number of the active drugs in top 4% ::: {m4} / {all_le} == {m4r}'),
                    pd.Series(f'the number of the active drugs in top 5% ::: {m5} / {all_le} == {m5r}'), 
                    pd.Series(f'the number of the active drugs in top 6% ::: {m6} / {all_le} == {m6r}'), 
                    pd.Series(f'the number of the active drugs in top 7% ::: {m7} / {all_le} == {m7r}'), 
                    pd.Series(f'the number of the active drugs in top 8% ::: {m8} / {all_le} == {m8r}'),
                    pd.Series(f'the number of the active drugs in top 9% ::: {m9} / {all_le} == {m9r}'),
                    pd.Series(f'the number of the active drugs in top 10% ::: {m10} / {all_le} == {m10r}')], ignore_index=True)
save_path = '../output/vs_crk12/'
os.makedirs(save_path, exist_ok=True)
new_csv.to_csv(f'{save_path}/vs_crk12.csv', encoding='gbk', index=False)
    
new_csv

,compound_id,pred_results,graph=smi+pocket_attention,graph=smi+pocket_in_protein_attention,sequence=smi+pocket_attention,sequence=smi+pocket_in_protein_attention,graph=smi_atominmol_indexes,graph=atom_in_mol+pocket_attention(atominmol_indexes),graph=atom_in_mol+protein_attention(atominmol_indexes),sequence=smi_subwinsmi_indexes,sequence=subword_in_smile+pocket_sequence_attention(subwinsmi_indexes),sequence=subword_in_smile+protein_sequence_attention(subwinsmi_indexes),pocinpro_indexes,0
0,C66200633101820074_1,5.131211,"[[-1.071001648902893, 1.0554227828979492, 1.24...","[[-1.2708230018615723, 1.1316097974777222, 1.2...","[[1.6654627323150635, -0.5504493117332458, 0.8...","[[1.52498197555542, -0.35180774331092834, 0.84...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-1.071001648902893, 1.0554227828979492, 1.24...","[[-1.9405150413513184, 1.6282575130462646, 1.8...","[0, 1, 2, 4, 5, 7, 9, 12, 16, 18, 20, 21, 22, ...","[[-0.5014050006866455, 0.916220486164093, 1.42...","[[-0.4918437600135803, 1.2569843530654907, 1.8...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",NaN
1,C4072998761844298_3,4.916657,"[[0.6650235056877136, -1.241023302078247, 0.24...","[[0.7888718843460083, -1.563041090965271, 0.24...","[[-1.4176968336105347, 0.16376259922981262, -1...","[[-1.0453062057495117, 0.34257981181144714, -1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.6650235056877136, -1.241023302078247, 0.24...","[[0.9646813273429871, -2.5452282428741455, 0.1...","[0, 1, 3, 6, 8, 9, 10, 12, 13, 14, 15, 17, 19,...","[[0.11105503886938095, -1.8402799367904663, 0....","[[0.2838352620601654, -2.223104953765869, 0.18...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",NaN
2,C5361960898898977_5,5.009614,"[[-0.30367326736450195, -1.9595056772232056, 1...","[[-0.41727834939956665, -2.289597511291504, 1....","[[-0.10449552536010742, -0.18062227964401245, ...","[[0.02076970785856247, -0.07776360958814621, -...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-0.30367326736450195, -1.9595056772232056, 1...","[[-0.6161109805107117, -3.3781321048736572, 1....","[0, 1, 2, 4, 5, 7, 10, 12, 13, 14, 15, 18, 20,...","[[-0.21361400187015533, -0.7828177809715271, 1...","[[-0.24111811816692352, -0.9871131181716919, 1...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",NaN
3,C3303408848892665_2,4.991557,"[[-0.8611473441123962, 0.7884159088134766, -0....","[[-1.0327354669570923, 0.8484691381454468, -0....","[[-0.19877664744853973, -1.2938512563705444, -...","[[0.03506040573120117, -0.9171868562698364, 0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[-0.8611473441123962, 0.7884159088134766, -0....","[[-1.648107886314392, 1.2088569402694702, -0.9...","[0, 1, 2, 3, 4, 5, 8, 10, 12, 13, 14, 16, 18, ...","[[-1.346024751663208, -0.3191666901111603, -1....","[[-1.4286047220230103, -0.07287472486495972, -...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",NaN
4,C5900654789368895,4.840687,"[[1.3212623596191406, 0.6185448169708252, -0.3...","[[1.4353508949279785, 0.5463617444038391, -0.6...","[[-0.09839213639497757, 0.8583430647850037, 0....","[[0.11765628308057785, 1.0238475799560547, 0.6...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[1.3212623596191406, 0.6185448169708252, -0.3...","[[2.1432044506073, 0.8038860559463501, -1.0247...","[0, 2, 4, 5, 6, 8, 9, 10, 12, 14, 16, 18, 20, ...","[[0.9864612817764282, -0.23631252348423004, -0...","[[1.4612209796905518, -0.24092228710651398, -0...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 6% ::: 8...
1169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 7% ::: 1...
1170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in top 8% ::: 1...
1171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the number of the active drugs in to

In [9]:
f'the number of the active drugs in top 1% ::: {m1} / {all_le} == {m1r}\
the number of the active drugs in top 2% ::: {m2} / {all_le} == {m2r}\
the number of the active drugs in top 3% ::: {m3} / {all_le} == {m3r}\
the number of the active drugs in top 4% ::: {m4} / {all_le} == {m4r}\
the number of the active drugs in top 5% ::: {m5} / {all_le} == {m5r}\
the number of the active drugs in top 6% ::: {m6} / {all_le} == {m6r}\
the number of the active drugs in top 7% ::: {m7} / {all_le} == {m7r}\
the number of the active drugs in top 8% ::: {m8} / {all_le} == {m8r}\
the number of the active drugs in top 9% ::: {m9} / {all_le} == {m9r}\
the number of the active drugs in top 10% ::: {m10} / {all_le} == {m10r}'

'the number of the active drugs in top 1% ::: 1 / 13 == 0.07692307692307693the number of the active drugs in top 2% ::: 2 / 13 == 0.15384615384615385the number of the active drugs in top 3% ::: 2 / 13 == 0.15384615384615385the number of the active drugs in top 4% ::: 3 / 13 == 0.23076923076923078the number of the active drugs in top 5% ::: 7 / 13 == 0.5384615384615384the number of the active drugs in top 6% ::: 8 / 13 == 0.6153846153846154the number of the active drugs in top 7% ::: 11 / 13 == 0.8461538461538461the number of the active drugs in top 8% ::: 11 / 13 == 0.8461538461538461the number of the active drugs in top 9% ::: 11 / 13 == 0.8461538461538461the number of the active drugs in top 10% ::: 11 / 13 == 0.8461538461538461'